In [1]:
import pandas as pd
import numpy as np
import scipy
import random

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [21]:
file_name = '../data/AdSmartABdata.csv'
ad_df = pd.read_csv(file_name)

In [22]:
ad_df.drop('Unnamed: 0',axis=1, inplace=True)
ad_df.reset_index(drop=True)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
1,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1,0
2,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,6,Chrome Mobile,0,1
3,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1,0
4,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,1
...,...,...,...,...,...,...,...,...,...
1238,ffa08ff9-a132-4051-aef5-01a9c79367bc,exposed,2020-07-05,21,Generic Smartphone,6,Chrome Mobile,1,0
1239,ffb176df-ecd2-45d3-b05f-05b173a093a7,exposed,2020-07-04,1,Generic Smartphone,6,Chrome Mobile,1,0
1240,ffb79718-6f25-4896-b6b3-e58b80a6e147,control,2020-07-09,7,Generic Smartphone,6,Chrome Mobile,0,1
1241,ffca1153-c182-4f32-9e90-2a6008417497,control,2020-07-10,16,Generic Smartphone,6,Chrome Mobile,0,1


### Split data by browser and platform_os, and version each split as a new version of the data in dvc.

#### Split the data into 70% training, 20% validation, and 10% test sets. 

#### Train a machine learning model using 5-fold cross validation using the following 3 different algorithms:
1. Logistic Regression 
2. Decision Trees
3. XGBoost


#### Define the appropriate loss function  for the model using the validation data

Compute feature importance

Which data features are relevant to predicting the target variable?

Which data features are relevant to predicting the target variable

For each of the ML algorithms above, find the best model by tuning their hyperparameters and each time adding the tried models in MLFlow

Prepare a Dockerfile for your project so that your model can be deployed in a docker container.